In [2]:
# データセットを読み込むためにpandasをインポートする
import pandas as pd

# pandasのread_csvメソッドを用いてトレーニングデータを読み込む
train_data = pd.read_csv('./train.tsv', delimiter='\t')

In [3]:
# 各特徴量の変換用の辞書を設定する
buying_mapping = {'vhigh': 4, 'high': 3, 'med': 2, 'low': 1}
maint_mapping = {'vhigh': 4, 'high': 3, 'med': 2, 'low': 1}
doors_mapping = {'5more': 5, '4': 4, '3': 3, '2': 2}
persons_mapping = {'more': 6, '4': 4, '2': 2}
lug_boot_mapping = {'big': 3, 'med': 2, 'small': 1}
safety_mapping = {'high': 3, 'med': 2, 'low': 1}

# クラスラベル（class）の変換用の辞書を設定する
class_mapping = {'vgood': 4, 'good': 3, 'acc': 2, 'unacc': 1}

# トレーニングデータの各特徴量を変換する
train_data['buying'] = train_data['buying'].map(buying_mapping)
train_data['maint'] = train_data['maint'].map(maint_mapping)
train_data['doors'] = train_data['doors'].map(doors_mapping)
train_data['persons'] = train_data['persons'].map(persons_mapping)
train_data['lug_boot'] = train_data['lug_boot'].map(lug_boot_mapping)
train_data['safety'] = train_data['safety'].map(safety_mapping)

# トレーニングデータのクラスラベル（class）を変換する
train_data['class'] = train_data['class'].map(class_mapping)

# 変換後のトレーニングデータを表示
train_data

,id,class,buying,maint,doors,persons,lug_boot,safety
0,0,1,1,2,3,2,1,1
1,3,2,1,3,3,6,1,2
2,7,1,4,3,5,2,1,2
3,11,2,3,3,3,6,3,2
4,12,1,3,3,3,2,2,3
...,...,...,...,...,...,...,...,...
859,1721,2,3,3,5,6,2,2
860,1722,1,3,3,5,4,1,2
861,1724,1,3,1,3,6,2,1
862,1725,1,1,1,5,2,1,2


In [5]:
# pandasのilocメソッドを用いてトレーニングデータを特徴量（train_data_X）とクラスラベル（train_data_Y）に分離する
# idは学習に用いないため別に保管する
train_data_X = train_data.iloc[:,2:]
train_data_Y = train_data.iloc[:,1:2]
train_data_id = train_data.iloc[:,0:1]

In [8]:
from sklearn.model_selection import train_test_split

# トレーニングデータを学習用データ（learning_data）と検証用データ（learning_data）に分割する
learning_data_X, validation_data_X, learning_data_Y, validation_data_Y = train_test_split(
    train_data_X, train_data_Y, test_size=0.3, random_state=1, stratify=train_data_Y)

In [9]:
# トレーニングデータを決定木モデルに適用する

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=1)
tree.fit(learning_data_X, learning_data_Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best')

In [13]:
tree.score(learning_data_X, learning_data_Y)

0.8394039735099338

In [10]:
tree.score(validation_data_X, validation_data_Y)

0.8692307692307693

In [11]:
# トレーニングデータをランダムフォレストモデルに適用する

from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='gini', n_estimators=25, random_state=1, n_jobs=2)
forest.fit(learning_data_X, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=2,
                       oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [14]:
forest.score(learning_data_X, learning_data_Y)

1.0

In [12]:
forest.score(validation_data_X, validation_data_Y)

0.9576923076923077

In [93]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1000.0, random_state=1)

# 標準化していない学習用データを用いてロジスティック回帰クラスを学習する
lr.fit(learning_data_X, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [94]:
lr.score(learning_data_X, learning_data_Y)

0.804635761589404

In [95]:
lr.score(validation_data_X, validation_data_Y)

0.7923076923076923

In [18]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()

stdsc.fit(learning_data_X)

# トレーニングデータの特徴量の標準化を行う
learning_data_X_std = stdsc.transform(learning_data_X)
validation_data_X_std = stdsc.transform(validation_data_X)

In [110]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)

# 標準化した学習用データを用いてロジスティック回帰クラスを学習する
lr.fit(learning_data_X_std, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [111]:
lr.score(learning_data_X_std, learning_data_Y)

0.8029801324503312

In [112]:
lr.score(validation_data_X_std, validation_data_Y)

0.7923076923076923

In [99]:
import numpy as np

cov_mat = np.cov(learning_data_X_std.T)

eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

In [100]:
# 固有値の大きさがほとんど均一なため主成分分析による次元削減は難しそう
eigen_vals

array([0.91961746, 0.96892591, 0.99406257, 1.02659414, 1.05590593,
       1.04484425])

In [101]:
from sklearn.decomposition import KernelPCA

scikit_kpca = KernelPCA(n_components=6, kernel='rbf', gamma=0.1)
scikit_kpca.fit(learning_data_X_std)

learning_data_X_std_kpca = scikit_kpca.transform(learning_data_X_std)
validation_data_X_std_kpca = scikit_kpca.transform(validation_data_X_std)

In [105]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=1000.0, random_state=1)

# 標準化+Kernel PCAによってデータ変換した学習用データを用いてロジスティック回帰クラスを学習する
lr.fit(learning_data_X_std_kpca, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1000.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [106]:
lr.score(learning_data_X_std_kpca, learning_data_Y)

0.8377483443708609

In [107]:
lr.score(validation_data_X_std_kpca, validation_data_Y)

0.8346153846153846

In [116]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)

# 標準化していない学習用データ（learning_data）を用いてロジスティック回帰クラスを学習する
lr.fit(learning_data_X, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [117]:
# 正解データ（validation_data）を用いて精度を求める
lr.score(validation_data_X, validation_data_Y)

0.7884615384615384

In [118]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()

stdsc.fit(learning_data_X)

# トレーニングデータの特徴量の標準化を行う
learning_data_X_std = stdsc.transform(learning_data_X)
validation_data_X_std = stdsc.transform(validation_data_X)

In [119]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)

# 標準化したトレーニングデータを用いてロジスティック回帰クラスを学習する
lr.fit(learning_data_X_std, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [120]:
lr.score(validation_data_X_std, validation_data_Y)

0.7923076923076923

In [123]:
import numpy as np

# 標準化したトレーニングデータ（learning_data_X_std）を用いて、共起行列を生成する
cov_mat = np.cov(learning_data_X_std.T)

# 固有値・固有ベクトルを計算する
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)

In [124]:
# 固有値の値を確認する
eigen_vals

array([0.91961746, 0.96892591, 0.99406257, 1.02659414, 1.05590593,
       1.04484425])

In [128]:
from sklearn.decomposition import KernelPCA

# カーネル主成分分析クラスのインスタンスを生成する
scikit_kpca = KernelPCA(n_components=6, kernel='rbf', gamma=15)

# 標準化したトレーニングデータ（learning_data_X_std）を用いて、カーネル行列を計算する
scikit_kpca.fit(learning_data_X_std)

# トレーニングデータと正解データを変換する
learning_data_X_std_kpca = scikit_kpca.transform(learning_data_X_std)
validation_data_X_std_kpca = scikit_kpca.transform(validation_data_X_std)

In [129]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1)

# 標準化+カーネル主成分分析によってデータ変換したトレーニングデータ（learning_data_X_std_kpca）を用いてロジスティック回帰クラスを学習する
lr.fit(learning_data_X_std_kpca, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [130]:
lr.score(validation_data_X_std_kpca, validation_data_Y)

0.6846153846153846

In [146]:
from sklearn.decomposition import KernelPCA

# カーネル主成分分析クラスのインスタンスを生成する
scikit_kpca = KernelPCA(n_components=6, kernel='rbf', gamma=0.1)

# 標準化したトレーニングデータ（learning_data_X_std）を用いて、カーネル行列を計算する
scikit_kpca.fit(learning_data_X_std)

# トレーニングデータと正解データを変換する
learning_data_X_std_kpca = scikit_kpca.transform(learning_data_X_std)
validation_data_X_std_kpca = scikit_kpca.transform(validation_data_X_std)

In [159]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=500.0, random_state=1)

# 標準化+カーネル主成分分析によってデータ変換したトレーニングデータ（learning_data_X_std_kpca）を用いてロジスティック回帰クラスを学習する
lr.fit(learning_data_X_std_kpca, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=500.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [161]:
# C=500.0、γ=0.1の場合の精度
lr.score(validation_data_X_std_kpca, validation_data_Y)

0.8384615384615385

In [162]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=1)

# 標準化していないトレーニングデータを用いて決定木を学習する
tree.fit(learning_data_X, learning_data_Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best')

In [163]:
tree.score(validation_data_X, validation_data_Y)

0.8692307692307693

In [164]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(criterion='gini', n_estimators=25, random_state=1, n_jobs=2)

# 標準化していないトレーニングデータを用いてランダムフォレストを学習する
forest.fit(learning_data_X, learning_data_Y)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=2,
                       oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [165]:
forest.score(validation_data_X, validation_data_Y)

0.9576923076923077